In [11]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
# import mlflow.sklearn

In [12]:
import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [13]:
np.random.seed(40)
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [14]:
# Read the wine-quality csv file from the URL
csv_url = (
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/data/winequality-red.csv"
)
try:
    data = pd.read_csv(csv_url, sep=";")
except Exception as e:
    logger.exception(
        "Unable to download training & test CSV, check your internet connection. Error: %s", e
    )

In [15]:
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

alpha = 0.3
l1_ratio = 0.8

In [18]:
exp_id = mlflow.get_experiment("229665458502710623")

In [19]:
# exp_id = mlflow.get_experiment_by_name("905852025789848453")
exp_id

<Experiment: artifact_location='file:///E:/Data%20Science%20projects/MLOps/getting%20started/mlruns/229665458502710623', creation_time=1678191195353, experiment_id='229665458502710623', last_update_time=1678191195353, lifecycle_stage='active', name='Test1', tags={}>

In [20]:
mlflow.get_experiment_by_name("Test1")

<Experiment: artifact_location='file:///E:/Data%20Science%20projects/MLOps/getting%20started/mlruns/229665458502710623', creation_time=1678191195353, experiment_id='229665458502710623', last_update_time=1678191195353, lifecycle_stage='active', name='Test1', tags={}>

In [43]:
mlflow.end_run()

In [45]:
lst_alpha=[0.8]
lst_l1_ratio=[0.8,0.85,0.9]

In [46]:
mlflow.set_experiment(experiment_name="Test1")
with mlflow.start_run():
#     for alpha in lst_alpha:
        for l1_ratio in lst_l1_ratio:
            
            lr=ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
            lr.fit(train_x, train_y)
            predicted_qualities = lr.predict(test_x)
            (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
            print("Elasticnet model (alpha={:f}, l1_ratio={:f}):".format(alpha, l1_ratio))
            print("  RMSE: %s" % rmse)
            print("  MAE: %s" % mae)
            print("  R2: %s" % r2)
#             with mlflow.start_run(nested=True):
#                 mlflow.log_param("alpha", alpha)
            with mlflow.start_run(nested=True):
                mlflow.log_param("l1_ratio", l1_ratio)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)
            urlparse(mlflow.get_tracking_uri()).scheme
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")
            else:
                mlflow.sklearn.log_model(lr, "model")            
    

Elasticnet model (alpha=0.800000, l1_ratio=0.800000):
  RMSE: 0.8330972295348316
  MAE: 0.669813814205792
  R2: 0.016611535037920344
Elasticnet model (alpha=0.800000, l1_ratio=0.850000):
  RMSE: 0.8330580589767124
  MAE: 0.6701120025768793
  R2: 0.0167040067641282
Elasticnet model (alpha=0.800000, l1_ratio=0.900000):
  RMSE: 0.8330212125502486
  MAE: 0.6704102143580977
  R2: 0.016790987837928095


'file'

C:\Users\Asus\anaconda3\envs\nlp_tasks\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
